In [10]:
import os
import torch
import math
import io
import os
import torch
from torchvision import transforms
from torchvision.utils import save_image
import numpy as np
import time
import onnx
import onnxruntime

from PIL import Image


In [11]:
path_to_model_export=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\pytorch_models\cheng2020_anchor.pth"
model_imported=torch.load(path_to_model_export,weights_only=False)
model_imported.eval()

Cheng2020Anchor(
  (entropy_bottleneck): EntropyBottleneck(
    (likelihood_lower_bound): LowerBound()
    (matrices): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 128x3x3 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 128x1x3 (cuda:0)]
    )
    (biases): ParameterList(
        (0): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (1): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (2): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (3): Parameter containing: [torch.float32 of size 128x3x1 (cuda:0)]
        (4): Parameter containing: [torch.float32 of size 128x1x1 (cuda:0)]
    )
    (factors): ParameterList(
        (0): Parameter containing: 

In [16]:
# Some dummy input
x = torch.randn(1, 3, 1232, 1640, requires_grad=True)

pytorch_models_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\pytorch_models"
onnx_models_export_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\onnx_models"




def export_pytorch_to_onnx(pytorch_path,onnx_path):

    #first load the pytorch model 
    model_imported=torch.load(pytorch_path,weights_only=False)
    model_imported.eval()

    print("Pytorch model is imported")

    
    # Export the model
    torch.onnx.export(model_imported,                       # model being run
                      x,                         # model input (or a tuple for multiple inputs)
                      onnx_path,              # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=14,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input': {0 : 'batch_size'},    # variable length axes
                                    'output': {0 : 'batch_size'}}
                     )
    print("Model is exported")


def test_export_by_loading(input_tensor_shape,onnx_model_export_path):
    """
    Just the 4 dimension is expected [ batch,channels,height,width]
    """
    
    onnx_model=onnx.load(onnx_model_export_path)
    onnx_model_graph=onnx_model.graph
    onnx_session=onnxruntime.InferenceSession(onnx_model.SerializeToString())

    x=torch.randn(input_tensor_shape).numpy()
    print("Size of x:",x.size)

    input_names=["input"]
    output_names=["output"]

    start=time.time()
    onnx_output=onnx_session.run(output_names,{input_names[0]:x})[0]
    end=time.time()
    print("Time for the inference is:",round(end-start,2),"seconds")
    
    

pytorch_model_name="bmshj2018-factorized_4_mse"

pytorch_model_path=os.path.join(pytorch_models_path,pytorch_model_name+".pth")
onnx_model_export_path=os.path.join(onnx_models_export_path,pytorch_model_name+".onnx")

export_pytorch_to_onnx(pytorch_model_path,onnx_model_export_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Swapnil\\Narrowband_DRONE\\Image_compression_code\\Final_DL_compession_September_24\\PyTorch_inference\\pytorch_models\\bmshj2018-factorized_4_mse.pth'